In [84]:
import numpy as np

def initialize_network(layers, neurons_per_layer):
    weights = []
    for layer in range(layers):
        layer_weights = [np.random.normal(0, 1, 3) for _ in range(neurons_per_layer)]
        weights.append(layer_weights)
    return weights

def forward_propagation(inputs, weights, neuron_idx=None):

    if neuron_idx is not None:
        a1 = inputs[neuron_idx % len(inputs)]
        a2 = inputs[(neuron_idx + 1) % len(inputs)]
    else:
        a1, a2 = inputs[0], inputs[1]

    f = np.array([
        a1 * a2,
        a1 + a2 - a1 * a2,
        a1 + a2 - 2 * a1 * a2
    ])

    exp_weights = np.exp(weights)
    p = exp_weights / np.sum(exp_weights)

    a_prime = np.sum(p * f)
    return p, f, a_prime

def cross_entropy_loss(a_prime, target):
    exp_a_prime = np.exp(a_prime)
    q1 = exp_a_prime / (1 + exp_a_prime)
    q0 = 1 / (1 + exp_a_prime)

    loss = -(target * np.log(q1) + (1 - target) * np.log(q0))
    return loss, q0, q1

def backward_propagation(p, f, q1, target, weights, learning_rate=0.1):
    dL_dq1 = q1 - target
    dq1_da_prime = q1 * (1 - q1)
    da_prime_dp = f
    dp_dweights = p * (1 - p)

    gradients = dL_dq1 * dq1_da_prime * da_prime_dp * dp_dweights

    updated_weights = weights - learning_rate * gradients
    return updated_weights

def aggregate_outputs(outputs, neurons_per_class):
    num_classes = len(outputs) // neurons_per_class
    aggregated_outputs = []
    for i in range(num_classes):
        aggregated_output = np.sum(outputs[i * neurons_per_class:(i + 1) * neurons_per_class])
        aggregated_outputs.append(aggregated_output)
    return aggregated_outputs

def train_network(inputs, targets, weights, layers, neurons_per_layer, neurons_per_class, epochs, learning_rate):
    for epoch in range(epochs):
        if (epoch+1)%1000 == 0:
          print(f"Epoch {epoch + 1}/{epochs}")

        correct_predictions = 0

        for data_idx, input_data in enumerate(inputs):
            if len(input_data) != 2:
                raise ValueError("Each input data point must have exactly 2 features.")

            layer_inputs = input_data
            all_layer_outputs = []

            for layer in range(layers):
                layer_outputs = []
                for neuron_idx in range(neurons_per_layer):
                    p, f, a_prime = forward_propagation(layer_inputs, weights[layer][neuron_idx], neuron_idx)
                    layer_outputs.append(a_prime)

                layer_inputs = layer_outputs
                all_layer_outputs.append(layer_outputs)

            aggregated_outputs = aggregate_outputs(all_layer_outputs[-1], neurons_per_class)
            predicted_class = np.argmax(aggregated_outputs)

            target = targets[data_idx]
            if predicted_class == target:
                correct_predictions += 1

            for layer in reversed(range(layers)):
                for neuron_idx in range(neurons_per_layer):
                    p, f, a_prime = forward_propagation(layer_inputs, weights[layer][neuron_idx], neuron_idx)
                    loss, q0, q1 = cross_entropy_loss(a_prime, target)
                    weights[layer][neuron_idx] = backward_propagation(p, f, q1, target, weights[layer][neuron_idx], learning_rate)

            if (epoch+1)%1000 == 0:
              print(f"  Data {data_idx + 1}: Loss = {loss:.4f}, Aggregated Outputs = {aggregated_outputs}")

        accuracy = correct_predictions / len(inputs)
        if (epoch+1)%1000 == 0:
          print(f"  Epoch {epoch + 1} Accuracy: {accuracy * 100:.2f}%")

    return weights

def main():
    inputs = [
        [0.6, 0.8],
        [0.5, 0.1],
        [0.4, 0.9]
    ]
    targets = [1, 0, 1]

    layers = 3
    neurons_per_layer = 6
    neurons_per_class = 3

    weights = initialize_network(layers, neurons_per_layer)

    epochs = 20000
    learning_rate = 0.99
    weights = train_network(inputs, targets, weights, layers, neurons_per_layer, neurons_per_class, epochs, learning_rate)


if __name__ == "__main__":
    main()


Epoch 1000/20000
  Data 1: Loss = 0.3800, Aggregated Outputs = [2.241493748523084, 2.3481197783826264]
  Data 2: Loss = 0.9048, Aggregated Outputs = [1.0532866096660811, 1.0859110992756276]
  Data 3: Loss = 0.3976, Aggregated Outputs = [2.080298035812442, 2.153858238557909]
  Epoch 1000 Accuracy: 66.67%
Epoch 2000/20000
  Data 1: Loss = 0.3980, Aggregated Outputs = [2.210902553649812, 2.0433796098163164]
  Data 2: Loss = 0.8254, Aggregated Outputs = [1.07987683226977, 0.7259726274795679]
  Data 3: Loss = 0.4184, Aggregated Outputs = [2.065260013280466, 1.8103533833286312]
  Epoch 2000 Accuracy: 33.33%
Epoch 3000/20000
  Data 1: Loss = 0.4167, Aggregated Outputs = [2.2101193227981324, 2.0419483946240735]
  Data 2: Loss = 0.7885, Aggregated Outputs = [1.075726271272849, 0.6545345823626353]
  Data 3: Loss = 0.4393, Aggregated Outputs = [2.079565706786124, 1.801135666931407]
  Epoch 3000 Accuracy: 33.33%
Epoch 4000/20000
  Data 1: Loss = 0.4187, Aggregated Outputs = [2.1921828788051516, 2.

In [71]:
    np.random.normal(0, 1, 3)

    print("q0: ", round(q0, 2))
    print("q1: ", round(q1, 2))

NameError: name 'q0' is not defined